In [29]:
import pandas as pd

In [30]:
from spotify.spotify_service import SpotifyManager
import os

def choose_playlist(spotify_manager):
    playlists = spotify_manager.get_user_playlists()
    for i, playlist in enumerate(playlists['items']):
        print(f"{i+1}: {playlist['name']}")

    choice = 1#int(input("Enter the number of the playlist you want to analyze: "))
    # choice=26
    return playlists['items'][choice - 1]['id']

def main():
    spotify_manager = SpotifyManager()
    playlist_id = choose_playlist(spotify_manager)
    print(playlist_id)
    full_track_data = spotify_manager.get_full_track_data(playlist_id)

    for track_data in full_track_data:
        track = track_data['track']
        print(f"{track['name']} by {track['artists'][0]['name']}")
        audio_features = track.get('audio_features')
        if audio_features:
            print(f"  - Tempo: {audio_features['tempo']}")
            print(f"  - Key: {audio_features['key']}")
            # Add more features as needed
    print(track_data)

In [32]:
import spotipy
sp = spotipy.Spotify(
            auth_manager=SpotifyOAuth(
                client_id=os.getenv("SPOTIPY_CLIENT_ID"),
                client_secret=os.getenv("SPOTIPY_CLIENT_SECRET"),
                redirect_uri=os.getenv("SPOTIPY_REDIRECT_URI"),
                scope=scope,
            )
        )
sp.current_user_playlists(limit=50)

NameError: name 'SpotifyOAuth' is not defined

In [ ]:
spotify_manager = SpotifyManager()
playlist_id = choose_playlist(spotify_manager)
full_track_data = spotify_manager.get_full_track_data(playlist_id)

In [ ]:
full_track_data[0].keys()

In [ ]:
full_track_data[0]['track'].keys()

In [ ]:
full_track_data[0]['track']['album_info']['release_date']

In [ ]:
def filter_track_data(track_data):
    # Initialize a new dictionary to hold the filtered data
    filtered_data = {}

    # Extract track information
    track = track_data.get('track', {})
    filtered_data['song_name'] = track.get('name')
    filtered_data['song_popularity'] = track.get('popularity')

    # Extract audio features
    audio_features = track.get('audio_features', {})
    filtered_data['audio_features'] = {key: audio_features[key] for key in audio_features if key not in ['type', 'id', 'uri', 'track_href', 'analysis_url']}

    # Extract artist information (assuming first artist)
    artist_info = track.get('artist_info', {})
    filtered_data['artist_name'] = artist_info.get('name')
    filtered_data['artist_popularity'] = artist_info.get('popularity')
    # Include artist genres
    filtered_data['artist_genres'] = artist_info.get('genres', [])

    # Extract album information
    album_info = track.get('album_info', {})
    filtered_data['album_release_date'] = album_info.get('release_date')
    filtered_data['album_popularity'] = album_info.get('popularity')

    return filtered_data


In [ ]:
def dict_to_dataframe(track_data):
    # Initialize a dictionary to hold the data for DataFrame creation
    df_data = {}

    # Add song name and artist name
    df_data['song_name'] = [track_data['song_name']]
    df_data['artist_name'] = [track_data['artist_name']]

    # Add numerical audio features
    for feature, value in track_data['audio_features'].items():
        df_data[feature] = [value]

    # Create a DataFrame
    df = pd.DataFrame(df_data)

    return df

In [ ]:
songs = []
for track_data in full_track_data:
    filtered_data = filter_track_data(track_data)
    songs.append(filtered_data)
    df = dict_to_dataframe(filtered_data)
    print(df.shape)


In [ ]:
len(songs)

In [ ]:
df


In [ ]:
filtered_data = filter_track_data(full_track_data[0])
filtered_data

In [ ]:
def dict_to_dataframe(track_data):
    # Initialize a dictionary to hold the data for DataFrame creation
    df_data = {}

    # Add song name and artist name
    df_data['song_name'] = [track_data['song_name']]
    df_data['artist_name'] = [track_data['artist_name']]

    # Add numerical audio features
    for feature, value in track_data['audio_features'].items():
        df_data[feature] = [value]

    # Create a DataFrame
    df = pd.DataFrame(df_data)

    return df

In [ ]:
dict_to_dataframe(filtered_data)

In [ ]:
def print_dict_keys(d, indent=0):
    for key, value in d.items():
        print(' ' * indent + str(key))
        if isinstance(value, dict):
            print_dict_keys(value, indent + 4)  # Increase indentation for nested dictionaries


In [ ]:
print_dict_keys(full_track_data[0])

In [ ]:
from utils.spotify_utils import choose_playlist, filter_track_data, dict_to_dataframe
import pandas as pd
from spotify.spotify_service import SpotifyManager
from clustering.song_clusterer import SongClusterer

spotify_manager = SpotifyManager()
playlist_id = '4Haf9xLfppVCbx3lcZvLUs'#choose_playlist(spotify_manager)
full_track_data = spotify_manager.get_full_track_data(playlist_id)

track_dfs = []
for track_data in full_track_data:
    if isinstance(track_data.get('track'), dict):
        filtered_dict = filter_track_data(track_data)
        track_df = dict_to_dataframe(filtered_dict)
        track_dfs.append(track_df)
    else:
        print(f"Skipping track due to unexpected data format: {track_data}")

# Only proceed if there are DataFrames to concatenate
if track_dfs:
    full_df = pd.concat(track_dfs, ignore_index=True)
    # Specify the features columns used for clustering

    # Define hyperparameters for DBSCAN
    hyperparams = {'eps': 0.9, 'min_samples': 2}

    # Create an instance of SongClusterer and apply clustering
    clusterer = SongClusterer(hyperparams)
    clustered_df = clusterer.apply_dbscan_clustering(full_df)

    # Analyze the clusters
    clusterer.analyze_clusters(clustered_df)

    # Optionally, save the clustered DataFrame to a new CSV
    clustered_df.to_csv('clustered_playlist_songs_info.csv', index=False)
else:
    print("No track data to save.")


In [ ]:
clustered_df

In [ ]:
from llm.llm_config import initialize_google_llm

llm = initialize_google_llm()


llm.invoke("hello")

In [ ]:

# from llm.llm_config import initialize_openai_llm

# llm = initialize_openai_llm()

# llm.invoke("hello")

In [ ]:
from llm.chains import build_analyser_chain


In [ ]:
chain = build_analyser_chain()

In [ ]:
inputs = {
    "songs": songs,
    "clustering_analysis": ""
}

response = chain.invoke(inputs)

In [ ]:
print(response.content)